# Target Accrual Redemption Forward (TARF) Barrier

This notebook demonstrates how to access and use the functionalities of **Target Accrual Redemption Forward (TARF)** which are part of our **QPS** module within LSEG Financial Analytics SDK.

**You will be able to:**
* Define TARF instrument with basic parameters
* Configure pricing parameters
* Evaluate TARF Analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp

import datetime as dt
import pandas as pd
import json
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example barrier TARF, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (underlying rate, payoff description, dates, notional)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing models and parameters, optional

In [2]:
# 1. Create SP definition object

print("Step 1: Configuring instrument definition...")

TARF_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "BarrierTARF",
    inputs = [
        sp.NameTypeValue(name="Underlying", type = "string", value="USDEUR"),
        sp.NameTypeValue(name="StartDate", type = "date", value= dt.date(2022, 1, 25)),
        sp.NameTypeValue(name="EndDate", type = "string", value="StartDate + 1Y"),
        sp.NameTypeValue(name="Frequency", type = "string", value="2M"),
        sp.NameTypeValue(name="Notional", type = "string", value="100000"),
        sp.NameTypeValue(name="Strike", type = "string", value="1.02"),
        sp.NameTypeValue(name="Barrier", type = "string", value="1.5"),
        sp.NameTypeValue(name="BarrierSide", type = "string", value="1"),
        sp.NameTypeValue(name="ProfitTarget", type = "string", value="100%"),
        sp.NameTypeValue(name="Leverage", type = "string", value="2"),
        sp.NameTypeValue(name="AppliedLeverage", type = "string", value="If(FX[t] <= Strike, 1, If(BarrierSide * FX[t] > BarrierSide * Barrier, Leverage, 0))"),
        sp.NameTypeValue(name="KO_Payment", type = "string", value="Settlement[t]"),
    ],
    payoff_description = [
					[
						"Schedule Type",
						"Schedule description",
						"FX",
						"Coupon",
						"Settlement",
						"Sum",
						"Alive",
						"KO_Amount",
						"Price"
					],
					[
						"AtDate",
						"StartDate",
						"",
						"",
						"0",
						"",
						"IF(ProfitTarget >= 0, 1, 0)",
						"",
						""
					],
					[
						"OnSchedule",
						"DateTable(StartDate + Frequency, EndDate, Frequency, ResetGap := 0b)",
						"FxSpot(Underlying)",
						"AppliedLeverage * (Strike - FX[t]) ",
						"Coupon[t] * Notional",
						"Sum[LastDate] + max(Coupon[t],0)",
						"If(Sum[t] >= ProfitTarget, 0, Alive[LastDate-1])",
						"Alive[LastDate-1] * (1-Alive[LastDate]) * KO_Payment",
						"Receive (Alive[t] * Settlement[t] + KO_Amount[t])"
					],
					[
						"AtDate",
						"EndDate",
						"FxSpot(Underlying)",
						"AppliedLeverage * (Strike - FX[t]) ",
						"Coupon[t] * Notional",
						"Sum[LastDate] + max(Coupon[t],0)",
						"If(Sum[t] >= ProfitTarget, 0, Alive[LastDate-1])",
						"Alive[LastDate-1] * (1-Alive[LastDate]) * KO_Payment",
						"Receive (Alive[t] * Settlement[t] + KO_Amount[t])"
					]
				]
)


print("	Instrument definition configured")

# 2. Create SP instrument definition object
print("Step 2: Creating instrument definition object...")

barrier_tarf = sp.StructuredProductsDefinitionInstrument(definition = TARF_definition)
print("	Instrument definition created")


# 3. Create SP parameters object - optional
print("Step 3: Configuring pricing parameters...")

TARF_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2022, 3, 16),  # Set your desired valuation date
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
            underlying_code = "USDEUR",
            underlying_tag = "USDEUR",
            underlying_currency = "EUR",
            asset_class = "ForeignExchange",
            model_name= "Heston")]
)
print("	Pricing parameters configured")

Step 1: Configuring instrument definition...
	Instrument definition configured
Step 2: Creating instrument definition object...
	Instrument definition created
Step 3: Configuring pricing parameters...
	Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price function
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = sp.price(
        definitions=[barrier_tarf],
        pricing_preferences=TARF_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing Execution Successful!")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Pricing Execution Successful!


## Results Display

### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., underlying, payoff description, dates, notional), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method, it is StructuredProductsPricingParameters we used

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` with discount factors
   - **description**: Instrument summary StructuredProductDefinition and also default fields not specified in the StructuredProductDefinition, but used by default in the calculation (e.g. price)
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

The analytics fields are useful for understanding which fields are included by default in the price function, even if they are not explicitly specified in `price` function. We detail below the description, valuation, cash-flows and greeks.

### Description

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "BarrierTARF",
    "dealCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 5.889430636,
        "Coupon": 0.734304457,
        "FX": 5.408588413,
        "KO_Amount": 2378.014131588,
        "Price": 72617.303989937,
        "Settlement": 73430.445680739,
        "Sum": 2.515874039
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}


### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_tarf_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_tarf_valuation)

,Field,Value
0,marketValueInDealCcy,72617.30399


### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Build dataframes for all cash flow types
df_fx = pd.DataFrame(cashflows[0]['payments']).rename(columns={'amount': cashflows[0]['legTag']})
df_coupon = pd.DataFrame(cashflows[1]['payments']).rename(columns={'amount': cashflows[1]['legTag']})
df_settlement = pd.DataFrame(cashflows[2]['payments']).rename(columns={'amount': cashflows[2]['legTag']})
df_sum  = pd.DataFrame(cashflows[3]['payments']).rename(columns={'amount': cashflows[3]['legTag']})
df_alive = pd.DataFrame(cashflows[4]['payments']).rename(columns={'amount': cashflows[4]['legTag']})
df_ko_amount = pd.DataFrame(cashflows[5]['payments']).rename(columns={'amount': cashflows[5]['legTag']})
df_price = pd.DataFrame(cashflows[6]['payments']).rename(columns={'amount': cashflows[6]['legTag']})

# Merge all dataframes on the 'date' column
# Add suffix to distinguish between different data sources
combined_df = pd.concat(
    [df_fx.add_suffix('_fx'), 
     df_coupon.add_suffix('_coupon'), 
     df_settlement.add_suffix('_settlement'), 
     df_sum.add_suffix('_sum'), 
     df_alive.add_suffix('_alive'), 
     df_ko_amount.add_suffix('_ko_amount'), 
     df_price.add_suffix('_price')], 
    axis=1
)

# Remove duplicate columns with the same values
combined_df = combined_df.loc[:, ~combined_df.T.duplicated()]

# Keep only one discountFactor column and sort the columns
combined_df = combined_df.loc[:,['date_fx', 'discountFactor_fx', 'FX_fx', 'Coupon_coupon','Settlement_settlement','Sum_sum',  'Alive_alive',
       'KO_Amount_ko_amount','Price_price', 'currency_fx', 'event_fx','occurence_fx']]

# Rename columns to match the desired layout
combined_df.columns = ['date', 'discountFactor', 'FX', 'Coupon', 'Settlement', 'Sum', 'Alive', 'KO_Amount', 'Price', 'currency', 'event', 'occurrence']

# Display the combined dataframe
display(combined_df)

,date,discountFactor,FX,Coupon,Settlement,Sum,Alive,KO_Amount,Price,currency,event,occurrence
0,2022-01-25,0.000000,0.000000,0.000000,0.000000,0.000000,1.0000,0.000000,0.000000,EUR,Payoff,Historical
1,2022-03-25,1.000123,0.906117,0.113883,11388.254333,0.113883,1.0000,0.000000,11399.563075,EUR,Payoff,Future
2,2022-05-25,1.001116,0.904112,0.116464,11646.359850,0.230346,1.0000,0.000000,11657.156394,EUR,Payoff,Future
3,2022-07-25,1.002044,0.901621,0.119924,11992.443699,0.350271,1.0000,0.000000,12001.865069,EUR,Payoff,Future
4,2022-09-26,1.002831,0.898307,0.123484,12348.406123,0.474703,0.9976,92.448429,12354.643409,EUR,Payoff,Future
5,2022-11-25,1.003338,0.895248,0.127539,12753.938276,0.603284,0.9752,660.952043,12665.147389,EUR,Payoff,Future
6,2023-01-25,1.003616,0.891465,0.131371,13137.105739,0.736381,0.9040,1616.301247,12377.982031,EUR,Payoff,Future


### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_tarf_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_tarf_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,-5125.2000
1,gammaAmountInDealCcy,-143.2180
2,vegaAmountInDealCcy,-711.4790
3,thetaAmountInDealCcy,-70.8949
